In [ ]:
from flask import Flask, request, jsonify
from flask_restful import Resource, Api, reqparse
from flask_jwt import JWT
from flask_jwt import jwt_required
from security import authenticate, identity
from user import UserRegister


app = Flask(__name__)
app.secret_key = 'tym'
api = Api(app)

jwt = JWT(app, authenticate, identity) # new endpoint /auth

items = []

class Item(Resource):
    @jwt_required()
    def get(self, name):
        item = next(filter(lambda x: x['name'] == name, items), None) # first item found
        return {'item': item}, 200 if item else 404
        
    def post(self, name):
        if next(filter(lambda x: x['name'] == name, items), None) is not None:
            return {'message': "An item with name '{}' already exists".format(name)}, 400
        
        data = self.validate_data('price','This field cannot be left blank').parse_args()
        
        new_item = {
                'name': name,
                'price': data['price']
        }
        items.append(new_item)
        return new_item, 201
    
    def delete(self, name):
        global items
        items = list(filter(lambda x: x['name'] != name, items)) # return list of items
        return {'message': "'{}' item deleted".format(name)}
    
    def put(self, name):
        data = self.validate_data('price','This field cannot be left blank').parse_args()
        
        item = next(filter(lambda x: x['name'] == name, items), None)
        if item is None:
            new_item = {
                'name': name,
                'price': data['price']
            }
            items.append(new_item)
        else:
            item.update(data)
        
        return item, 201
    
    
    def validate_data(self, field, message):
        parser = reqparse.RequestParser()
        parser.add_argument(field, 
                            type=float, 
                            required=True, 
                            help=message
        )
        return parser
    
class ItemList(Resource):
    def get(self):
        return {'items': items}

api.add_resource(Item, '/item/<string:name>') # http://127.0.0.1/itemt/<item name>
api.add_resource(ItemList, '/items')
api.add_resource(UserRegister, '/register')
app.run(port=5001)

 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jan/2018 12:56:58] "POST /register HTTP/1.1" 400 -
